# Installations

In [1]:
%pip install --upgrade pip --quiet
%pip install pandas --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install langchain_openai --quiet
!pip install --upgrade langchain_community --quiet
!pip install eventregistry --quiet


[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  DEPRECATION: eventregistry is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
import requests
import pandas as pd

from eventregistry import *

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI

# Broad Picture

We have a higher level agent at the top - let's call it **Fin-Agent** - it has access to following

**SQL Agent**

SQL agent has access to a database contained stock details, fundamentals, important ratios and technicals. 

The agent is responsible for converting natural language questions to sql code using the table schema

**News Agent**

News agents is responsible to formualte the right keywords which will be used to fetch relevant stock news (companny and industry)
The agent is also expected to summarise the new articles into provide a concise report to the Fin-Agent

**Insider transactions tool**

**Current stock data tool**

Using these tools fin-agent determines whether to hold or sell a stock. The idea is - fin agent runs through all the stocks held by an individual or an organisation and suggests whether they should sell or hold the stock they have already bought. It could also run through entire list of stocks and determine whether to buy, sell or hold a stock too. 



## Note: We only have data till 2022-09 in the database. 

## Creating SQL agent

In [58]:
# Load the configuration from the JSON file
with open('config_4o.json', 'r') as config_file:
    config = json.load(config_file)

# Extract the OpenAI key and endpoint from the configuration
openai_key = config["AZURE_OPENAI_KEY"]
azure_endpoint = config["AZURE_OPENAI_ENDPOINT"]
model_name = config["CHATGPT_MODEL"]
api_version = config["AZURE_OPENAI_APIVERSION"]

db_loc = 'stock_db.db'

db = SQLDatabase.from_uri(f"sqlite:///{db_loc}")

gpt4o_azure = AzureChatOpenAI(
    model_name=model_name,
    api_key=openai_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    temperature=0
)


database_schema = db.get_table_info()
# print(database_schema)

system_message = f"""current date is 2022-09-30. You are a SQLite expert agent designed to interact with a SQLite database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 100 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database..
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Only use the following tables:
{database_schema}
"""

full_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", '{input}'),
        MessagesPlaceholder("agent_scratchpad")
    ]
)

sqlite_agent = create_sql_agent(
    llm=gpt4o_azure,
    db=db,
    prompt=full_prompt,
    agent_type="openai-tools",
    agent_executor_kwargs={'handle_parsing_errors':True},
    max_iterations=5,
    verbose=True
)

In [59]:
@tool
def sql_tool(user_input):
    """
    Executes a SQL query using the sqlite_agent and returns the result.
    Args:
        user_input (str): The SQL query to be executed.
    Returns:
        str: The result of the SQL query execution. If an error occurs, the exception is returned as a string.
    """
    try:
        response = sqlite_agent.invoke(user_input)

        prediction = response['output']

    except Exception as e:
        prediction = e
        
    return prediction

In [60]:
# ## 1
user_input = "What are the fundamentals of apple - ticker AAPL for today?"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT "date", "ttm_net_eps", "pe_ratio", "ttm_sales_per_share", "book_value_per_share", "ttm_fcf_per_share", "current_assets", "current_ratio", "current_assets_inventory", "quick_ratio", "total_liabilities", "debt_to_equity_ratio", "return_on_equity", "return_on_assets", "return_on_investment", "tangible_equity", "return_on_tangible_equity", "price", "current_liabilities", "shareholders_equity", "ttm_net_income", "cash_on_hand", "total_share_holder_equity", "revenue", "gross_profit", "operating_income", "ebitda", "net_income", "eps_earnings_per_share_diluted", "shares_outstanding", "ps_ratio", "pb_ratio", "pfcf_ratio", "total_assets" FROM fundamentals WHERE "ticker" = \'AAPL\' AND "date" = \'2022-09-30\''}`


```sql
SELECT "date", "ttm_net_eps", "pe_ratio", "ttm_sales_per_share", "book_value_per_share", "ttm_fcf_per_share", "current_assets", "current_ratio", "current_assets_inventory", "qui

In [25]:
## 2
user_input = "Provide me the financial information of microsoft like revenue profit and EBIDTA for today"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "revenue", "gross_profit", "operating_income", "ebitda", "net_income", "eps_earnings_per_share_diluted", "shares_outstanding" FROM "revenue_profit" WHERE "ticker" = \'MSFT\' AND "date" = \'2022-09-30\' LIMIT 100;'}`


[('MSFT', '2022-09-30', 50122.0, 34670.0, 21518.0, 24308.0, 17556.0, 2.35, 7485.0)]
Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" FROM "assets_liabilities" WHERE "ticker" = \'MSFT\' AND "date" = \'2022-09-30\' LIMIT 100;'}`


[('MSFT', '2022-09-30', 359784.0, 107262.0, 45374.0, 186218.0, 173566.0)]
Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "ttm_net_eps", "pe_ratio", "ttm_sales_per_share", "book_value_per_share", "ttm_fcf_per_share", "current_assets", "current_ratio", "quick_ratio", "total_liabil

In [3]:
user_input = "What are all the company stock values present in the database. provide me the unique companies (ticker) in the price table?"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT DISTINCT "ticker" FROM "price";'}`


```sql
SELECT DISTINCT "ticker" FROM "price";
```
Invoking: `sql_db_query` with `{'query': 'SELECT DISTINCT "ticker" FROM "price";'}`


[('AAPL',), ('ABBV',), ('ABT',), ('ACN',), ('ADBE',), ('AMZN',), ('ASML',), ('AVGO',), ('AZN',), ('BABA',), ('BAC',), ('BMY',), ('BRK.A',), ('CMCSA',), ('COP',), ('COST',), ('CSCO',), ('CVX',), ('DHR',), ('DIS',), ('GOOGL',), ('HD',), ('JNJ',), ('JPM',), ('KO',), ('LIN',), ('LLY',), ('MA',), ('MCD',), ('META',), ('MRK',), ('MSFT',), ('NEE',), ('NKE',), ('NVDA',), ('NVS',), ('ORCL',), ('PEP',), ('PFE',), ('PG',), ('PM',), ('RTX',), ('SCHW',), ('SHEL',), ('TMO',), ('TMUS',), ('TSLA',), ('TTE',), ('TXN',), ('UNH',), ('UPS',), ('V',), ('VZ',), ('WFC',), ('WMT',), ('XOM',)]The unique companies (tickers) present in the price table are:

- AAPL
- ABBV
- ABT
- ACN
- ADBE
- AMZN
- ASML
- AVGO
- AZN
- BABA
- BAC
- BMY
- BRK.

In [61]:
## 3
user_input = "Which is the industry pe value for apple?"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT "pe_ratio" FROM industry_pe_ratios WHERE "industry" = (SELECT "industry" FROM industry_ticker_mapping WHERE "ticker" = \'AAPL\')'}`


```sql
SELECT "pe_ratio" FROM industry_pe_ratios WHERE "industry" = (SELECT "industry" FROM industry_ticker_mapping WHERE "ticker" = 'AAPL')
```
Invoking: `sql_db_query` with `{'query': 'SELECT "pe_ratio" FROM industry_pe_ratios WHERE "industry" = (SELECT "industry" FROM industry_ticker_mapping WHERE "ticker" = \'AAPL\')'}`


[(32.28,)]The industry PE ratio for Apple (AAPL) is 32.28.

> Finished chain.
The industry PE ratio for Apple (AAPL) is 32.28.


## Creating News Researcher Agent

When we tried to create seperate tools for news, they overloadede the top-level agent with a lot of information. To over-ride this we created a seperate agent for these tools that captures news from them and then summarises it and sends it up to the top-level agent. We found this a nice, practical case where we need multiple agents in a system instead of a lot of tools with a single agent.

We will start with creating tools required by this agent

Get free api key from https://newsapi.ai/

### Create tools

In [97]:
with open('financial_apis.json', 'r') as fin_config:
    fin_config = json.load(fin_config)

In [98]:
apha_api_key = fin_config['apha_api_key']
os.environ["HUGGINGFACE_API_KEY"] = fin_config["HF_token"]
news_client_id = fin_config["news_client_id"]

In [99]:
er = EventRegistry(apiKey = news_client_id, allowUseOfArchive=False)

'nytimes.com'

In [229]:
@tool
def company_news(company_name: str) -> list:
    """
    Searches for a collection of news articles [most relevant, most socially shared and keyword based news] based on the given company name and industry keywords, and returns the news context as a list of articles.

    Args:
        company_name (str): The name of the company for finding relevant news articles.
        industry_keywords (list): A list of keywords related to the industry.

    Returns:
        list: A list of dictionaries containing the news articles.
    """
    news = []

    # Get most relevant news articles based on the company name
    q_pos = QueryArticlesIter(
        conceptUri=er.getConceptUri(company_name),
        categoryUri=er.getCategoryUri("investing"),
        dataType=["news"],
        lang="eng"
    )
    for art in q_pos.execQuery(er, sortBy="cosSim", maxItems=10):
        news.append(art)
    return news

@tool
def industry_news(industry_keywords: list) -> list:
    """
    Fetches the most relevant articles related to a given list of industry keywords.
    Args:
        industry_keywords (list): A list of keywords representing the industry of interest.
    Returns:
        list: A list of news articles related to the provided industry keywords.

    """
    news = []

    q_pos = QueryArticlesIter(
        conceptUri=QueryItems.OR([er.getConceptUri(i) for i in industry_keywords]),
        categoryUri=er.getCategoryUri("investing"),
        dataType=["news"],
        lang = "eng"
    )
    for art in q_pos.execQuery(er, sortBy="cosSim", maxItems=10): #sort by options - 
        news.append(art)

    return news



**parameter options for execQuery**

- **eventRegistry**: Instance of the `EventRegistry` class used to obtain the necessary data.

- **sortBy**: Order in which event articles are sorted. Options include:
        - `none`: No specific sorting
        - `id`: Internal ID
        - `date`: Published date
        - `cosSim`: Closeness to event centroid
        - `sourceImportance`: Manually curated score of source importance (high value, high importance)
        - `sourceImportanceRank`: Reverse of `sourceImportance`
        - `sourceAlexaGlobalRank`: Global rank of the news source
        - `sourceAlexaCountryRank`: Country rank of the news source
        - `socialScore`: Total shares on social media
        - `facebookShares`: Shares on Facebook only

- **sortByAsc**: Determines if the results should be sorted in ascending (`True`) or descending (`False`) order.

- **returnInfo**: Specifies what details should be included in the returned information.

- **maxItems**: Maximum number of items to be returned. Used to stop iteration sooner than results run out.

In [ ]:
# Testing the news_tool

In [ ]:
company_name = "Apple inc."
keywords = ["computers", "electronics"]

# Use the invoke method instead of __call__
response = industry_news.invoke({"industry_keywords": keywords})
response

[{'uri': '2024-09-477496679',
  'lang': 'eng',
  'isDuplicate': False,
  'date': '2024-09-06',
  'time': '14:44:45',
  'dateTime': '2024-09-06T14:44:45Z',
  'dateTimePub': '2024-09-06T14:42:39Z',
  'dataType': 'news',
  'sim': 0.9882352948188782,
  'url': 'https://www.streetinsider.com/Business+Wire/Calyxia+raises+%2435+million+to+help+global+manufacturers+advance+product+performance%2C+and+fight+microplastic+pollution+and+GHG+emissions/23696488.html',
  'title': 'Calyxia raises $35 million to help global manufacturers advance product performance, and fight microplastic pollution and GHG emissions',
  'body': 'Investment led by Lombard Odier Investment Managers\' Plastic Circularity Fund and Bpifrance, through its Large Venture fund, bringing cumulative funding to $70M.\n\nPARIS--(BUSINESS WIRE)-- Calyxia, an industrial B2B Cleantech innovator, certified B Corp, specializing in advanced and biodegradable microcapsules and microparticles for use in Consumer Goods, Crop Protection and Ad

In [ ]:
response = company_news.invoke({"company_name": company_name})
response

[{'uri': '8337944028',
  'lang': 'eng',
  'isDuplicate': False,
  'date': '2024-09-26',
  'time': '10:31:01',
  'dateTime': '2024-09-26T10:31:01Z',
  'dateTimePub': '2024-09-26T10:30:16Z',
  'dataType': 'news',
  'sim': 0.9921568632125854,
  'url': 'https://www.theglobeandmail.com/investing/article-power-thirst-complicates-esg-investors-love-affair-with-tech-stocks/',
  'title': "Power thirst complicates ESG investors' love affair with tech stocks",
  'body': 'Investors who manage hundreds of billions of dollars are pressing Microsoft, Alphabet and others for more information on the power needed for artificial intelligence and advanced computing, to help decide whether the sector should stay heavily represented in sustainable funds, investors said.\n\nWhile these conversations are at an early stage, six fund industry executives in Europe and the United States said they are looking more closely at the environmental impact of the AI boom, which Goldman Sachs estimates will boost data cen

### Creating the News Agent using the two tools

In [230]:
system_message = f"""
You are a financial researcher who can look through a lot of news articles and extract very important insights for a news article.
You have access to tools that can give you news articles based on the company name and also based on the industry keywords.
your job is to give the right company name and pick the right industry keywords to get the most relevant news articles.

After fetching the news articles - go through them and extract the most important insights from the news articles and summarize them in bullet points. 
You can choose to not use a tool and reply 'I am not able to determine whether to hold it or not. Here is the information I do have: ' along with summary of all the information you have gathered if the tools are not useful to answer the question.
or If answer is not available in the tool output - do the same and reply with the summary of all the information you have with a note.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [company_news, industry_news]

news_llm = AzureChatOpenAI(
    model_name=model_name,
    api_key=openai_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-02-01",
    temperature=0
)

news_agent = create_tool_calling_agent(news_llm, tools, prompt)
news_agent_executor = AgentExecutor(agent=news_agent, tools=tools, verbose=True)


In [231]:
# news_agent_executor.invoke({"input": "apple inc."})



> Entering new AgentExecutor chain...

Invoking: `company_news` with `{'company_name': 'Apple Inc.'}`


[{'uri': '2024-09-496969714', 'lang': 'eng', 'isDuplicate': False, 'date': '2024-09-26', 'time': '10:31:12', 'dateTime': '2024-09-26T10:31:12Z', 'dateTimePub': '2024-09-26T06:04:00Z', 'dataType': 'news', 'sim': 0.9921568632125854, 'url': 'https://www.usnews.com/news/top-news/articles/2024-09-26/analysis-power-thirst-complicates-esg-investors-love-affair-with-tech-stocks', 'title': "Analysis-Power Thirst Complicates ESG Investors' Love Affair With Tech Stocks", 'body': 'NEW YORK (Reuters) - Investors who manage hundreds of billions of dollars are pressing Microsoft, Alphabet and others for more information on the power needed for artificial intelligence and advanced computing, to help decide whether the sector should stay heavily represented in sustainable funds, investors said.\n\nWhile these conversations are at an early stage, six fund industry executives in Europe and the United

{'input': 'apple inc.',
 'output': 'Here are the most important insights from the recent news articles related to Apple Inc. and the technology industry:\n\n### Apple Inc. Specific Insights:\n1. **Environmental Impact and ESG Concerns**:\n   - Investors managing hundreds of billions of dollars are pressing tech giants, including Apple, for more information on the power needed for AI and advanced computing.\n   - Concerns are rising about the environmental impact of the AI boom, which is expected to boost data center power demand by 160% by 2030.\n   - Apple, along with other tech companies, has started reporting increased greenhouse gas emissions due to the power and materials needed for data centers.\n   - Despite these concerns, none of the investors contacted are considering divesting from these companies.\n\n2. **Sustainability and Renewable Energy**:\n   - Apple has been part of the top holdings in ESG-focused funds, but there is pressure to maintain commitments to renewable energ

### Convert the agent into a tool that our top-level agent can use

In [234]:
@tool
def news_tool(company_name:str) -> str:
    """
    Fetches the most relevant news articles based on the company name.
    It automatically fetched the most relevant keywords and extracts for the industry and then gets news related to those keywords.

    Args:
        company_name (str): The name of the company for finding relevant news articles.

    Returns:
        str: The most important insights extracted from the news articles.
    """
    try:
        response = news_agent_executor.invoke({"input": company_name})
        response = response["output"]
    except Exception as e:
        response = e
    return response

In [237]:
news_tool.invoke({"company_name":"Nvidia"})



> Entering new AgentExecutor chain...

Invoking: `company_news` with `{'company_name': 'Nvidia'}`


[{'uri': '2024-09-496969714', 'lang': 'eng', 'isDuplicate': False, 'date': '2024-09-26', 'time': '10:31:12', 'dateTime': '2024-09-26T10:31:12Z', 'dateTimePub': '2024-09-26T06:04:00Z', 'dataType': 'news', 'sim': 0.9921568632125854, 'url': 'https://www.usnews.com/news/top-news/articles/2024-09-26/analysis-power-thirst-complicates-esg-investors-love-affair-with-tech-stocks', 'title': "Analysis-Power Thirst Complicates ESG Investors' Love Affair With Tech Stocks", 'body': 'NEW YORK (Reuters) - Investors who manage hundreds of billions of dollars are pressing Microsoft, Alphabet and others for more information on the power needed for artificial intelligence and advanced computing, to help decide whether the sector should stay heavily represented in sustainable funds, investors said.\n\nWhile these conversations are at an early stage, six fund industry executives in Europe and the United Sta

"Here are the most important insights from the recent news articles about Nvidia and the semiconductor industry:\n\n### Nvidia-Specific Insights:\n1. **Environmental Concerns and ESG Investments**:\n   - Investors managing hundreds of billions of dollars are pressing Nvidia and other tech giants for more information on the power needed for AI and advanced computing.\n   - Increased greenhouse gas emissions from data centers are raising questions from asset managers who want their portfolios to perform well financially and environmentally.\n   - Nvidia, along with other tech giants, is heavily represented in sustainable funds, but this could change if environmental concerns are not addressed.\n\n2. **Stock Market Performance**:\n   - Nvidia's stock has experienced significant volatility. For instance, Nvidia's shares tumbled nearly 10% in early September, contributing to a broader market sell-off.\n   - Despite recent losses, Nvidia's stock is still up 118% year-to-date, indicating stro

## Create tools to fetch insider transactions and recent stock prices

Get free api key from https://www.alphavantage.co/support/#api-key

In [213]:
date= "2022-09-30"

@tool
def get_stock_price(ticker:str):
    """
    Fetches the stock price for a given ticker symbol using the Alpha Vantage API.

    Args:
        ticker (str): The ticker symbol of the stock.

    Returns:
        dict: A dataframe containing the latest stock data with 30min interval.
    """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval=30min&month=2022-09&outputsize=full&apikey={apha_api_key}'
    r = requests.get(url)
    data = r.json()

    
    # Extract the time series data
    time_series = data['Time Series (30min)']

    # Convert the time series data into a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Rename the columns
    df.columns = ['open', 'high', 'low', 'close', 'volume']

    # Convert the index to datetime format
    df.index = pd.to_datetime(df.index)

    # Filter for the latest date
    latest_date = df.index.max().date()
    filtered_df = df[df.index.date == latest_date]

    return filtered_df

@tool
def insider_transactions(ticker:str):
    """
    Fetches the insider transactions data for a given company ticker symbol using the Alpha Vantage API.

    Args:
        ticker (str): The ticker symbol of the company.

    Returns:
        dict: A dictionary containing the insider transactions data of the company.

    Raises:
        requests.exceptions.RequestException: If there is an issue with the HTTP request.
        ValueError: If the response cannot be parsed as JSON.
    """
    url = f'https://www.alphavantage.co/query?function=INSIDER_TRANSACTIONS&symbol={ticker}&apikey={apha_api_key}'
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data["data"])

    # Convert the 'shares' column to numeric type
    df['shares'] = pd.to_numeric(df['shares'])

    # Sort the DataFrame by the number of shares in descending order
    df_sorted = df.sort_values(by='shares', ascending=False)

    # Select the top 10 transactions
    top_10_transactions = df_sorted.head(10)


    return top_10_transactions


@tool
def catchall(input: str):
    """
    A catchall function when other tools are not applicable
    """

    return "I am not able to determine whether to hold it or not. Here is the information I do have: " + input



In [214]:
data = insider_transactions.invoke({"ticker": "NVDA"})
data

,transaction_date,ticker,executive,executive_title,security_type,acquisition_or_disposal,shares,share_price
3805,2016-03-16,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,3078820.0,0.0
3811,2016-03-16,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,3078820.0,0.0
7,2024-09-20,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,1568000.0,0.0
3035,2019-05-24,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,A,1251950.0,0.0
3036,2019-05-24,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,A,1251950.0,0.0
1820,2021-11-23,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,1218536.0,0.0
1818,2021-11-23,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,1218536.0,0.0
1653,2022-09-06,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,A,1200000.0,3.4275
1652,2022-09-06,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Employee Stock Option (Right to Buy),D,1200000.0,0.0
933,2024-04-22,NVDA,"NVIDIA, CORP",10% Owner,Common Stock,A,1050129.0,2.42


## Create Fin-Agent

In [240]:
system_message = f"""
You are an expert stock trader equipped with several tools to analyze a company's financial health, stock performance, and market sentiment. 
Your task is to determine whether we should sell a specific stock based on comprehensive data analysis.
Before you start functioning, stop for a minute and think about all the tools you have and how you can use them to make an informed decision.
come up with a step by step plan (keep the plan brief - 3-4 words pointers is enough) to make a decision on whether to hold or sell the stock.
Answer only with the output from tools. Note that the tool selection is optional. However, be thorough with your analysis - use all the tools at your disposal to make an informed decision.

The sql tool has access to all the following information and can answer any question using the database:
{database_schema}. you can ask questions in natural language and the tool will convert them into SQL queries to fetch the required information.
when using the sql_tool make sure you are asking for the most recent data. send instruction to get data by ordering the table by data and picking the latest date from top unles you specifically need data from other dates.

You can use the different news tools to gather different news articles [relevant to company, and news about the industry keywords (figure out 2-3 industry keywords  based on the company profile)] and summarise them before coming to your conclusion
The next instruction is very impoertant: As you accrue data - keep summarizing the information within 10000 words to make a decision on whether to hold or sell the stock.
Remember to summarise the information you have gathereed at every step.Do not re-iterate your plan at every step.
You can choose to not use a tool and reply 'I am not able to determine whether to hold it or not. Here is the information I do have: ' along with summary of all the information you have gathered if the tools are not useful to answer the question.
or If answer is not available in the tool output - do the same and reply with the summary of all the information you have with a note.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [sql_tool,get_stock_price,insider_transactions,news_tool, catchall]

llm = AzureChatOpenAI(
    model_name=model_name,
    api_key=openai_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-02-01",
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent.get_prompts()

# # Tests
# ## 

# query = "Should we sell or hold the microsoft stock?"
# response = agent_executor.invoke({"input": query})
# print(response['output'])

[ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessa

## Testing Fin-Agent

In [197]:
query = "Should we sell or hold the johnson and johnson stock?"
response = agent_executor.invoke({"input": query})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_tool` with `{'user_input': "SELECT * FROM industry_ticker_mapping WHERE ticker = 'JNJ'"}`




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT * FROM industry_ticker_mapping WHERE ticker = 'JNJ'"}`


```sql
SELECT * FROM industry_ticker_mapping WHERE ticker = 'JNJ'
```
Invoking: `sql_db_query` with `{'query': "SELECT * FROM industry_ticker_mapping WHERE ticker = 'JNJ'"}`


[('JNJ', 'Drug Manufacturers - General')]The ticker 'JNJ' belongs to the industry 'Drug Manufacturers - General'.

> Finished chain.
The ticker 'JNJ' belongs to the industry 'Drug Manufacturers - General'.
Invoking: `sql_tool` with `{'user_input': "SELECT * FROM fundamentals WHERE ticker = 'JNJ' ORDER BY date DESC LIMIT 1"}`




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT ticker, date, ttm_net_eps, pe_ratio, ttm_sales_per_share, book_value_per_share, ttm_

In [239]:
query = "Should we sell or hold the nvidia stock?"
response = agent_executor.invoke({"input": query})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_tool` with `{'user_input': "SELECT * FROM assets_liabilities WHERE ticker = 'NVDA' ORDER BY date DESC LIMIT 1;"}`




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" FROM "assets_liabilities" WHERE "ticker" = \'NVDA\' ORDER BY "date" DESC LIMIT 1;'}`


```sql
SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" 
FROM "assets_liabilities" 
WHERE "ticker" = 'NVDA' 
ORDER BY "date" DESC 
LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" FROM "assets_liabilities" WHERE "ticker" = \'NVDA\' ORDER BY "da

In [241]:
query = "Should we sell or hold the Charles Schwab Corporation stock?"
response = agent_executor.invoke({"input": query})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_tool` with `{'user_input': "SELECT * FROM assets_liabilities WHERE ticker = 'SCHW' ORDER BY date DESC LIMIT 1"}`




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity FROM assets_liabilities WHERE ticker = 'SCHW' ORDER BY date DESC LIMIT 1"}`


```sql
SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity 
FROM assets_liabilities 
WHERE ticker = 'SCHW' 
ORDER BY date DESC 
LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': "SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity FROM assets_liabilities WHERE ticker = 'SCHW' ORDER BY date DESC LIMIT 1"}`


[('SCHW', '2022-09-30', 577563.0, 90604.0, 2